# Data Visualisation

The goal of this notebook is to visualise the Mars fault line labels, which are in the form of a shapefile.

The extra challenge of this is the fact that the geometry of the shapefile is a Martian geometry (MARS2000). This geometry is projected, and so the geometry is in x,y coordinate pair format.

Ideally this visualisation is in the form of an interactive map on top of a Martian basemap. Even more ideally, this basemap corresponds to the map used to derive the fault lines in the first place so that everything lines up nicely.

## 1. Config

In [ ]:
import geopandas as gpd
import folium
import shapely
import os
import pyproj
import pycrs

## 2. Data Loading

In [ ]:
labels = gpd.read_file("gs://esg-satelite-data-warehouse/mars/mars_labels/Orlov_TempeTerra_fault_catalogue.shp")

In [ ]:
labels.head(1)

## 3. Plotting Using Folium

In [ ]:
os.environ["PROJ_IGNORE_CELESTIAL_BODY"] = "YES"

The following code cell is not used but here for reference.

- We can load in the projection file associated with the shapefile we have and create a transformer.
- This transformer allows us to convert the x,y coordinate pairs into lat/long coordinates (or back if we wish).
- The problem with this is that the lat/long pairs are **Martian** lat/long pairs. A particular latitude on Mars is a difference distance from the meridian when compared to the same latitude on Earth, due to differences in planet diameter.
- Therefore this approach is not useful.

In [ ]:
# crs = pycrs.load.from_file("projection.prj")
# proj4 = crs.to_proj4()
# p = pyproj.Proj(proj4)

# labels["geometry"] = labels.geometry.map(lambda polygon: shapely.ops.transform(lambda x, y: p(x, y, inverse=True), polygon))

We need to convert our Martian data into an Earth-based projection. This is because our visualisation tooling only supports Earth projections.

The goal therefore is to map the Martian points that we have onto an Earth-sized sphere. Thankfully, this is actually quite straightforward. We use the EPSG:4236 projection here which is in lat/long format.

In [ ]:
labels = labels.to_crs("epsg:4326")

In [ ]:
labels.head(1)

Now we can produce the map.

- The basemap we are using is from ArcGIS: https://www.arcgis.com/home/item.html?id=07b33bb3067747abbebd1ce34c29816e
- This is a colourised basemap, actually derived from this data: https://astrogeology.usgs.gov/search/map/Mars/Topography/HRSC_MOLA_Blend/Mars_HRSC_MOLA_BlendDEM_Global_200mp_v2

It turns out, this data is ultimately one of the features used to derive the fault lines in the first place. So this is ideal.

We also need to make sure we set the CRS to be the same as our data (4326) and that of the basemap (?).

In [ ]:
Map = folium.Map(
    tiles="https://astro.arcgis.com/arcgis/rest/services/OnMars/MColorDEM/MapServer/tile/{z}/{y}/{x}.png", 
    crs="EPSG4326",
    attr="Basemap from: https://www.openplanetary.org/opm/basemaps"
)

Setting an appropriate default zoom.

In [ ]:
x1, y1, x2, y2 = labels["geometry"].total_bounds
Map.fit_bounds([[y1, x1], [y2, x2]])

And adding our fault lines.

In [ ]:
_ = folium.GeoJson(labels["geometry"]).add_to(Map)

In [ ]:
Map